In [70]:
from telethon import TelegramClient, events, sync
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel

import pandas as pd

from googletrans import Translator
from afinn import Afinn

In [71]:
params = pd.read_csv('./params_tg.csv', sep=';')

In [72]:
api_id = params['api_id'].values[0]
api_hash = str(params['api_hash'].values[0])

phone = '+' + str(params['phone'].values[0])
username = params['username'].values[0]

client = TelegramClient('session_name', api_id, api_hash)
await client.start()
print("Client Created")
client.run_until_disconnected()

Client Created


<coroutine object UpdateMethods._run_until_disconnected at 0x136708e40>

In [73]:
channels = await client.get_dialogs()
news_channels = ['Коммерсантъ'] # список каналов, из которых мы хотим получать новости
news_channel_ids = []
for channel in channels:
    if channel.title in news_channels:
        news_channel_ids.append(channel.id)

In [74]:
channel_source = dict(zip(news_channel_ids, news_channels))

In [75]:
news = []
for channel_id in news_channel_ids:
    channel = PeerChannel(channel_id)
    messages = await client(GetHistoryRequest(
        peer=channel,
        limit=100, # количество сообщений, которые мы хотим получить
        offset_date=None,
        offset_id=0,
        max_id=0,
        min_id=0,
        add_offset=0,
        hash=0))
    for message in messages.messages:
        news.append({
            'date': message.date,
            'text': message.message,
            'source': channel_id
        })

In [76]:
df = pd.DataFrame(news)

In [78]:
client.disconnect()

<Future pending cb=[shield.<locals>._outer_done_callback() at /Users/kirillkuznecov/opt/anaconda3/envs/ds/lib/python3.11/asyncio/tasks.py:898]>

In [80]:
translator = Translator(service_urls=['translate.google.com'])
afinn = Afinn()

In [81]:
def score_func(news_text):

    def translate(text):
        translator = Translator(service_urls=['translate.google.com'])
        translation = translator.translate(text, dest='en').text
        return translation

    def score(translation):
        return afinn.score(translation)

    scoring = 0

    try:
        scoring = score(translate(news_text))
    except TypeError:
        pass

    return scoring


In [82]:
df['source'] = df['source'].map(channel_source)
df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))
df['date'] = df['date'].dt.tz_localize(None)
df['score'] = df['text'].apply(score_func)

In [95]:
try:
    main_df = pd.read_excel('./TG_news.xlsx', index_col=0)
    main_df = pd.concat([main_df, df], axis=0, ignore_index=True)
    main_df.drop_duplicates(inplace=True)

    with pd.ExcelWriter('./TG_news.xlsx') as writer:
        main_df.to_excel(writer, sheet_name='1')

except FileNotFoundError:
    with pd.ExcelWriter('./TG_news.xlsx') as writer:
        df.to_excel(writer, sheet_name='1')